# Lonboard Example

In [29]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import shapely
from palettable.colorbrewer.diverging import BrBG_10
from sidecar import Sidecar

from lonboard import Map, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap

In [30]:
url = "https://ookla-open-data.s3.us-west-2.amazonaws.com/parquet/performance/type=mobile/year=2019/quarter=1/2019-01-01_performance_mobile_tiles.parquet"

In [31]:
local_path = Path("./assets/internet-speeds.parquet")

if local_path.exists():
    gdf = gpd.read_parquet(local_path)
else:
    # Create a data frame
    columns = ["avg_d_kbps", "tile"]
    df = pd.read_parquet(url, columns = columns)

    # Create tiles and centroids
    tile_geometries = shapely.from_wkt(df["tile"])
    tile_centroids = shapely.centroid(tile_geometries)

    # Create a geo data frame and pass it to a Parquet file
    gdf = gpd.GeoDataFrame(df[["avg_d_kbps"]], geometry = tile_centroids, crs = "EPSG:4326")
    gdf.to_parquet(local_path)

In [32]:
gdf

,avg_d_kbps,geometry
0,5983,POINT (-160.01862 70.63722)
1,3748,POINT (-160.04059 70.63357)
2,3364,POINT (-160.04059 70.63175)
3,2381,POINT (-160.0351 70.63357)
4,3047,POINT (-160.0351 70.63175)
...,...,...
3231240,19528,POINT (169.81842 -46.29571)
3231241,15693,POINT (169.81293 -46.3071)
3231242,26747,POINT (169.66461 -46.42082)
3231243,67995,POINT (169.65912 -46.4511)


In [33]:
sidecar = Sidecar()

In [34]:
type(gdf)

geopandas.geodataframe.GeoDataFrame

In [35]:
# Create a scatterplot layer from GeoPandas
layer = ScatterplotLayer.from_geopandas(gdf)

# Configure the layer
layer.get_fill_color = [0, 0, 200, 200]
min_bound = 5000
max_bound = 50000
download_speed = gdf["avg_d_kbps"]
normalized_download_speed = (download_speed - min_bound) / (max_bound - min_bound)
normalized_download_speed
BrBG_10.mpl_colormap
layer.get_fill_color = apply_continuous_cmap(normalized_download_speed, BrBG_10, alpha = 0.7)
layer.get_radius = normalized_download_speed * 200
layer.radius_units = "meters"
layer.radius_min_pixels = 0.5

# Create a Lonboard map
m = Map(layer, height = 800)
# with Sidecar:
# display(m)
m

NameError: name 'ArrowDtype' is not defined